# Sorting Results

The following notebook describes how to:

1. Create a sample index consisting of fictitious hotel data
1. Sort the results by geographic distance

## Prerequisites
* An Azure subscription, with access to Azure OpenAI.
* Azure AI Search basic or higher for this workload
* A deployment of the text-embedding-3-large model on Azure OpenAI.

## Set up a Python virtual environment in Visual Studio Code
* Open the Command Palette (Ctrl+Shift+P).
* Search for Python: Create Environment.
* Select Venv.
* Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see Python environments in VS Code.

### 1. Install packages

In [ ]:
! pip install -r requirements.txt --quiet

### 2. Load .env file (Copy .env-sample to .env and update accordingly)

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
# You do not need a key if you are using keyless authentication
# To learn more, please visit https://learn.microsoft.com/azure/search/search-security-rbac
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# Data is pre-vectorized using text-embedding-3-large
model_name = "text-embedding-3-large"
aoai_embedding_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "text-embedding-3-large")
# You do not need a key if you are using keyless authentication
# To learn more, please visit https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources and https://learn.microsoft.com/azure/developer/ai/keyless-connections
aoai_key = os.getenv("AZURE_OPENAI_KEY", None)

### 3. Create search index

In [2]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    ComplexField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)
from azure.search.documents.indexes import SearchIndexClient


index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="HotelId", type="Edm.String", key=True, hidden=False, filterable=True, sortable=False, facetable=False, searchable=True),
        SearchField(name="HotelName", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
        SearchField(name="Description", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
        SearchField(name="DescriptionEmbedding", type="Collection(Edm.Single)", hidden=False, searchable=True, vector_search_dimensions=3072, vector_search_profile_name="hnsw"),
        SearchField(name="Description_fr", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True, analyzer_name="fr.microsoft"),
        SearchField(name="Description_fr_Embedding", type="Collection(Edm.Single)", hidden=False, searchable=True, vector_search_dimensions=3072, vector_search_profile_name="hnsw"),
        SearchField(name="Category", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
        SearchField(name="Tags", type="Collection(Edm.String)", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
        SearchField(name="ParkingIncluded", type="Edm.Boolean", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
        SearchField(name="LastRenovationDate", type="Edm.DateTimeOffset", hidden=False, filterable=False, sortable=True, facetable=False, searchable=False),
        SearchField(name="Rating", type="Edm.Double", hidden=False, filterable=True, sortable=True, facetable=True, searchable=False),
        ComplexField(name="Address", collection=False, fields=[
            SearchField(name="StreetAddress", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
            SearchField(name="City", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="StateProvince", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="PostalCode", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="Country", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True)
        ]),
        SearchField(name="Location", type="Edm.GeographyPoint", hidden=False, filterable=True, sortable=True, facetable=False, searchable=False),
        ComplexField(name="Rooms", collection=True, fields=[
            SearchField(name="Description", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
            SearchField(name="Description_fr", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True, analyzer_name="fr.microsoft"),
            SearchField(name="Type", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="BaseRate", type="Edm.Double", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
            SearchField(name="BedOptions", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="SleepsCount", type="Edm.Int64", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
            SearchField(name="SmokingAllowed", type="Edm.Boolean", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
            SearchField(name="Tags", type="Collection(Edm.String)", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
        ])
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw", vectorizer_name="openai", algorithm_configuration_name="hnsw")],
        algorithms=[HnswAlgorithmConfiguration(name="hnsw")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="openai",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=aoai_endpoint,
                    deployment_name=aoai_embedding_deployment,
                    model_name=model_name,
                    api_key=aoai_key,
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint, credential)
result = index_client.create_or_update_index(index)
print("Created sample index")

Created sample index


## 4. Upload sample documents

Upload the sample hotel documents to the index


In [3]:
from azure.search.documents import SearchClient
import json

client = SearchClient(endpoint, index_name, credential)
with open("../../../data/hotels.json", encoding="utf-8", mode="r") as f:
    documents = json.load(f)
    client.upload_documents(documents)
    print("Uploaded sample documents")

Uploaded sample documents


### 5. Sort by Distance

The following example query shows how to sort by distance to a geography point

More info here: https://learn.microsoft.com/en-us/azure/search/search-query-odata-orderby

In [6]:
from azure.search.documents.models import VectorizableTextQuery
query = "hotel in new york with pool and gym"
coordinates = [-73.935242, 40.730610]  # New York City coordinates

results = client.search(
    search_text=query,
    vector_queries=[VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="DescriptionEmbedding")],
    select="HotelId,HotelName,Description,Location",
    top=5,
    order_by=f"geo.distance(Location, geography'POINT({coordinates[0]} {coordinates[1]})') asc",
)

for result in list(results):
    print(f"HotelId: {result['HotelId']}, HotelName: {result['HotelName']}, Description: {result['Description']}")
    print(f"Location: {result['Location']['coordinates']}")
    print()

HotelId: 1, HotelName: Stay-Kay City Hotel, Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Location: [-73.975403, 40.760586]

HotelId: 15, HotelName: By the Market Hotel, Description: Book now and Save up to 30%. Central location. Walking distance from the Empire State Building & Times Square, in the Chelsea neighborhood. Brand new rooms. Impeccable service.
Location: [-73.989792, 40.756729]

HotelId: 17, HotelName: City Skyline Antiquity Hotel, Description: In vogue since 1888, the Antiquity Hotel takes you back to bygone era. From the crystal chandeliers that adorn the Green Room, to the arched ceilings of the Grand Hall, the elegance of old New York beckons. Elevate Your Experience. Upgrade to a premiere city